In [1]:
import tensorflow as tf
import os
import re
# import soxr
import pandas as pd
from scipy.io import wavfile
import sys
import numpy as np
import pickle
# import soundfile as sf
import argparse

sys.path.append('/om2/user/amagaro/spatial_audio_pipeline/utils')
import util_audio
import util_tfrecords

2025-05-09 16:03:31.388915: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-09 16:03:32.486680: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-09 16:03:36.423665: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [14]:
def write_config_pkl_files(example,
                           dir_dst,
                           fn_feature_description='config_feature_description.pkl',
                           fn_bytes_description='config_bytes_description.pkl'):
    """
    Write `feature_description` and `bytes_description` config dictionaries (which
    desribe contents of tfrecords for parsing) to files in destination directory.
    """
    feature_description, bytes_description = util_tfrecords.get_description_from_example(example)
    for k in sorted(bytes_description.keys()):
        if k.startswith('list_'):
            bytes_description[k]['shape'] = f'len_{k}'
    if not os.path.isabs(fn_feature_description):
        fn_feature_description = os.path.join(dir_dst, fn_feature_description)
    if not os.path.isabs(fn_bytes_description):
        fn_bytes_description = os.path.join(dir_dst, fn_bytes_description)
    with open(fn_feature_description, 'wb') as f:
        pickle.dump(feature_description, f)
        print(f'[WROTE CONFIG] `{fn_feature_description}`')
        for k in sorted(feature_description.keys()):
            print(f'|__ {k}: {feature_description[k]}')
    with open(fn_bytes_description, 'wb') as f:
        pickle.dump(bytes_description, f)
        print(f'[WROTE CONFIG] `{fn_bytes_description}`')
        for k in sorted(bytes_description.keys()):
            print(f'|__ {k}: {bytes_description[k]}')
    return


def get_word(filename):
    if '--' in filename:
        return filename.replace('--', '').split('-')[1]
    return filename.split('-')[1]


def get_word2(filename):
    return filename.split('_')[1][:-4]


def get_word3(filename):
    '''chimera fine structure condition'''
    return filename.split('_')[-3]

def get_word4(filename):
    '''fernanda code'''
    return filename.split('.')[0].lower()


def remove_punctuation(word, punctuation="\.,'-"):
    regex = '|'.join(list(punctuation))
    return re.sub(regex, '', word)


def sample_from_middle(array, num_samples):
    middle = len(array) // 2
    start_ix = middle - (num_samples // 2)
    return array[start_ix : start_ix + num_samples]


def format_example(src_dir, filename, word_int, arg_dict, sr=20000, reversed=True, num_seconds=2, condition='control', folder='control', snr=None):
    if snr is None:
        noise_level = 0
    else:
        noise_level = 60.0 - snr
    num_samples = sr * num_seconds
    setting = arg_dict[folder](filename)
#     print(src_dir + filename)
    src_sr, audio = wavfile.read(src_dir + filename)
    if audio.shape[0] < num_seconds * src_sr:
        audio = np.hstack([audio, np.zeros((num_seconds * src_sr - audio.shape[0],))])
    #print(sum(audio))
    #print('cp1', util_audio.get_dBSPL(audio), src_sr, audio, audio.shape)
    audio = sample_from_middle(scipy.signal.resample(audio, sr*num_seconds).astype(np.float32), num_samples)
    #print('cp2', util_audio.get_dBSPL(audio))
    audio = util_audio.set_dBSPL(audio, 60.0)
    gaussian_noise = np.random.normal(size=audio.shape)
    gaussian_noise = util_audio.set_dBSPL(gaussian_noise, noise_level) * (0 if not snr else 1)
#     print(gaussian_noise)
    audio = np.array(audio + gaussian_noise).astype(np.float32)
    example = {"label_word_int": word_int, "signal": audio, "sr": sr, "setting": setting}
    return example


def map_words_to_ints(df_manifest):
    word_dict = {}
    words = df_manifest["word"]
    word_ints = df_manifest["label_word_int"]
    for word, word_int in zip(words, word_ints):
        word_dict[remove_punctuation(word)] = word_int
    return word_dict


def get_indices(meta):
    # 1 if synth, 0 if original
    ind_dict = {}
    for ii, entry in enumerate(meta['path'].astype(str)): 
        if entry.split('/')[7].split('_')[-1] =='synth.wav':
            ind_dict[ii] = 1
        else:
            ind_dict[ii] = 0
    return ind_dict


def main(src_dir, dir_dst, filename, folder, condition, word_getter=get_word, is_speech=True, subcondition='coch', snr=None):
    dir_manifest = '/om2/user/msaddler/spatial_audio_pipeline/assets/dataset_word_recognition/manifest_foreground_speech_train.pdpkl'
    df_manifest = pd.read_pickle(dir_manifest)
    word_dict = map_words_to_ints(df_manifest)
    meta_josh_speech_quilts = np.load('/om2/user/ershook/spring_18/datasets/sound_quilts/meta/sound_quilt_meta.npy', allow_pickle=True)
    quilts_formatted = np.concatenate(([meta_josh_speech_quilts['folder'].astype(str)], [meta_josh_speech_quilts['path'].astype(str)], [ [entry.split('_')[-3] for entry in  meta_josh_speech_quilts['path'].astype(str)]]), axis = 0).T
    mm_inds = {cond: get_indices(np.load(f'/om2/user/ershook/spring_18/datasets/sam_model_matched_{cond}/meta/model_matched_{cond}_meta.npy')) for cond in ['coch', 'specmod', 'spectempmod', 'tempmod']}
    for i in quilts_formatted:
        if i[2] == 'Orig':
            i[2] = '0'
    # dictionary of functions to get the setting from the file name according to the condition
    arg_dict = {'cognitive_restoration': lambda x: int(x.split('_')[-2]),
                'compressed_dilated': lambda x: float(x.split('_')[-1][:-4]),
                'whispered': lambda x: 0 if x.split('_')[-1][:-4] == 'harm' else 1,
                'noise_vocoded': lambda x: int(x.split('_')[-2]),
                'inharmonic': lambda x: int(x.split('_')[-1][-6:-4]),
                'reverb': lambda x: int(x.split('_')[-2][1:]),
                'sine_wave_speech': lambda x: int(x.split('_')[-2]),
                'shuffled': lambda x: float(x.split('_')[-2]),
                'silenced': lambda x: float(x.split('_')[-2]),
                'masked': lambda x: int(x.split('_')[-2]),
                'repackaged': lambda x: float(x.split('_')[-2]),
                'clipped': lambda x: float(x.split('_')[-2]),
                'bandpass': lambda x: int(x.split('_')[-2]),
                'core': lambda x: 1,
                'mod': lambda x: int(x.split('_')[-1].split('-')[0])*(1 if x.split('_')[-2] == 'specmod' else -1),
                'mod_core': lambda x: float(x.split('_')[-1].split('-')[0])*(1 if x.split('_')[-2] == 'specmod' else -1),
                'mod_lowpass': lambda x: float(x.split('_')[-1].split('.')[0])*(1 if x.split('_')[-3] == 'specmod' else 1),
                'chimera': lambda x: int(x.split('_')[-2]),
                'chimera_noise': lambda x: int(x.split('_')[-3]),
                'lowpass': lambda x: int(x.split('_')[-3]),
                'highpass': lambda x: int(x.split('_')[-3]),
                'synth_reverb': lambda x: int(x.split('_')[-2][:-2]),
                'synth_reverb_babble_0dB': lambda x: int(x.split('_')[-2][:-2]),
                'backgrounds': lambda x: int(x.split('_')[-1][:-6]) if x.split('_')[-1][:-6] != 'inf' else 6,
                '43_textures': lambda x: int(x.split('_')[-1][:-4]),
                'tone_vocoded': lambda x: int(x.split('_')[-1][:-6]),
                'natural_sounds': lambda x: int(x.split('_')[0][4:]),
                'quilted': lambda x: int(quilts_formatted[int(x[4:8])][2]),
                'mm': lambda x: mm_inds[subcondition][int(x[4:8])],
                'control': lambda x: 0,
                'clean': lambda x: 0}
    fn_dst = dir_dst + filename
    config_written = False
    with tf.io.TFRecordWriter(fn_dst + '~OPEN', options='GZIP') as writer:
        for ix, filename in enumerate(os.listdir(src_dir)):
            if filename[-4:] != '.wav':
                continue
            if condition == 'quilted' and ix >= len(quilts_formatted):
                break
            # if '9_' not in filename and '0_' not in filename and '18_' not in filename:
            #     continue
            #if filename != 'Babble2Spkr_3db-people-13-11.1-wsj1-si_tr_s-4az-4azc030s_1.5_compressed_dilated_1.5.wav':
            #    continue
            word = remove_punctuation(word_getter(filename))
            if word in word_dict or not is_speech:
                word_int = word_dict[word] if is_speech else 0
                example = format_example(src_dir, filename, word_int, arg_dict, condition=condition, folder=folder, snr=snr)
                print(example['signal'].shape)
                #print(filename)
                writer.write(util_tfrecords.serialize_example(example))
                if not config_written:
                    write_config_pkl_files(example, dir_dst)
                    config_written = True
            if ix%100 == 0:
                print(ix)
        print(ix)
    # tfrecords file is closed and renamed
    os.rename(fn_dst + '~OPEN', fn_dst)

In [12]:
import scipy
int_job = 0
snr = None

In [15]:
job_list = ['clean', 'cognitive_restoration', 'compressed_dilated', 'whispered', 'noise_vocoded', 'inharmonic', # 0-5
                'reverb-0', 'reverb-1', 'reverb-2', 'sine_wave_speech', 'shuffled', 'silenced-0.25', 'silenced-0.5', # 6-12
                'silenced-0.75', 'masked-1', 'masked-3', 'masked-10', 'masked-30', 'masked-100', 'repackaged', # 13-19
                'clipped-peak', 'clipped-center', 'bandpass', 'mod_lowpass-specmod', 'mod_lowpass-tempmod', # 20-24
                'chimera-fs', 'chimera-envelope', 'chimera_noise-fs', 'lowpass', 'highpass', # 25-29
                'synth_reverb-10', 'synth_reverb-20', 'synth_reverb-30', 'synth_reverb-40', 'synth_reverb-50', # 30-34
                'synth_reverb-60', 'synth_reverb-70', 'synth_reverb_babble_0dB-20', 'synth_reverb_babble_0dB-50', # 35-38
                'backgrounds-audscene', 'backgrounds-babble', 'backgrounds-music', 'backgrounds-stat_noise', 'backgrounds-mod_noise', # 39-43
                *[f'43_textures-{i}' for i in ['neg9','neg6','neg3',0,3,'inf']],  # 44-49
                'tone_vocoded-stat_0', 'tone_vocoded-stat_8', 'tone_vocoded-stat_16', 'tone_vocoded-stat_24', 'tone_vocoded-stat_32', # 50-54
                'tone_vocoded-mod_0', 'tone_vocoded-mod_8', 'tone_vocoded-mod_16', 'tone_vocoded-mod_24', 'tone_vocoded-mod_32', # 55-59
    ]
for int_job in [1, 2, 3, 4, 5, 9, 10, 22, 23, 24, 28, 29, 31, 34, 39, 40, 41, 42, 43]:
    condition = job_list[int_job]
    print(condition)
    #src_dir = '/om2/user/msaddler/spatial_audio_pipeline/assets/human_experiment_v00/foreground_swc/'
    #src_dir = '/om/user/alexkell/psychophysics_stim_2016X/2017W-word-forNetwork/selected-stim-with-NO-bg/'
    #src_dir = '/om4/group/mcdermott/user/ershook/StimulusSetsFromOM2/165_natural_sounds/naturalsounds165/'
    src_superdir = '/om/scratch/Wed/amagaro/apmth220_project/wavs/'
    if condition == 'clean':
        src_dir = src_superdir + '/clean/'#f'/om2/user/amagaro/spatial_audio_pipeline/wavs/source_files/'
    elif condition == 'clipped-peak':
        src_dir = f'{src_superdir}/peak_clipping/'
    elif condition == 'clipped-center':
        src_dir = f'{src_superdir}/center_clipping/'
    elif '-' in condition and condition.split('-')[0] == 'chimera':
        src_dir = f'{src_superdir}/chimera/'
    elif '-' in condition:
        src_dir = f"{src_superdir}/{condition.split('-')[0]}/{condition.split('-')[1]}/"
    else:
        src_dir = f'{src_superdir}/{condition}/'

    if '-' in condition:
        folder = condition.split('-')[0]
    else:
        folder = condition

    if condition == 'chimera-fs':
        word_getter = get_word3
    else:
        word_getter = get_word
    #src_dir = '/om2/user/dlatorre/RobustRewrite/SynthesisSpeech/SpeechifyAudios_2seconds/'
    #src_dir = f'/om2/user/amagaro/spatial_audio_pipeline/wavs/source_files/'
    #src_dir = '/om2/user/ershook/spring_18/datasets/josh_speech_quilts_22/wavs/'
    #src_dir = '/om2/user/ershook/spring_18/datasets/sam_model_matched_'+subcond+'_jan22/wavs/'
    #test_name = 'Babble2Spkr_3db-people-13-11.1-wsj1-si_tr_s-4az-4azc030s_1.5_compressed_dilated_1.5.wav'
    #test_name2 = 'AudScene_neg3db-coming-13-11.1-wsj1-si_tr_s-4b0-4b0c020o_0.875_compressed_dilated_0.875.wav'
    dir_dst = f'/om/scratch/Wed/amagaro/apmth220_project/tfrecords/{folder}/'
    #dir_dst = '/om2/user/amagaro/spatial_audio_pipeline/generate_scenes/test/'
    filename = f'{condition}.tfrecords'
    print(src_dir)
    try:
        os.mkdir(dir_dst)
    except:
        print(f'{dir_dst} exists')
    main(src_dir, dir_dst, filename, folder=folder, condition=condition, word_getter=word_getter, is_speech=True, snr=snr)

cognitive_restoration
/om/scratch/Wed/amagaro/apmth220_project/wavs//cognitive_restoration/
/om/scratch/Wed/amagaro/apmth220_project/tfrecords/cognitive_restoration/ exists
(40000,)
[WROTE CONFIG] `/om/scratch/Wed/amagaro/apmth220_project/tfrecords/cognitive_restoration/config_feature_description.pkl`
|__ label_word_int: FixedLenFeature(shape=[], dtype=tf.int64, default_value=None)
|__ setting: FixedLenFeature(shape=[], dtype=tf.int64, default_value=None)
|__ signal: FixedLenFeature(shape=[], dtype=tf.string, default_value=None)
|__ sr: FixedLenFeature(shape=[], dtype=tf.int64, default_value=None)
[WROTE CONFIG] `/om/scratch/Wed/amagaro/apmth220_project/tfrecords/cognitive_restoration/config_bytes_description.pkl`
|__ signal: {'dtype': dtype('float32'), 'shape': (40000,)}
0
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)

(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
900
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000

(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
1800
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(4000

(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
2700
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(4000

(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
3600
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(4000

...

/tmp/ipykernel_984859/4038740816.py:68: WavFileWarning: Chunk (non-data) not understood, skipping it.
  src_sr, audio = wavfile.read(src_dir + filename)


(40000,)
[WROTE CONFIG] `/om/scratch/Wed/amagaro/apmth220_project/tfrecords/backgrounds/config_feature_description.pkl`
|__ label_word_int: FixedLenFeature(shape=[], dtype=tf.int64, default_value=None)
|__ setting: FixedLenFeature(shape=[], dtype=tf.int64, default_value=None)
|__ signal: FixedLenFeature(shape=[], dtype=tf.string, default_value=None)
|__ sr: FixedLenFeature(shape=[], dtype=tf.int64, default_value=None)
[WROTE CONFIG] `/om/scratch/Wed/amagaro/apmth220_project/tfrecords/backgrounds/config_bytes_description.pkl`
|__ signal: {'dtype': dtype('float32'), 'shape': (40000,)}
0
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40

(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
900
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000

(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
1800
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(4000

(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
2700
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(4000

In [2]:
import numpy as np
import json

In [3]:
src_dir = '/om2/user/amagaro/word_recognition_project/saved_models/'
# tester = np.load(src_dir + 'no_augs_400/arch0_0000/EVAL_220_mod_lowpass-specmod_activation_block1_relu.npy')

In [4]:
tester.shape

(3265, 25, 4969, 64)

In [4]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.8, svd_solver='full')

In [6]:
for model in ['no_augs_400', 'no_backgrounds', '100_talkers', 'extreme_augs_300']:
    tester = np.load(src_dir + f'{model}/arch0_0000/EVAL_220_all_clean_activation_block6_relu.npy')
    reduced = pca.fit_transform(np.mean(tester, axis=-1).reshape((tester.shape[0], np.prod(tester.shape[1:3]))))
    print(reduced.shape)

(653, 71)
(653, 36)
(653, 46)
(653, 61)


In [18]:
for model in ['no_augs_400', 'no_backgrounds', 'extreme_augs_300']:
    tester = np.load(src_dir + f'{model}/arch0_0000/EVAL_220_lowpass_activation_block6_relu.npy')
    with open(src_dir + f'{model}/arch0_0000/EVAL_220_lowpass.json') as f:
        params = json.load(f)['setting']
    indices = {param: np.array(params) == param for param in set(params)}
    for param, inds in indices.items():
        reduced = pca.fit_transform(np.mean(tester[inds], axis=-1).reshape((tester[inds].shape[0], np.prod(tester[inds].shape[1:3]))))
        print(model, param, reduced.shape)

no_augs_400 400 (653, 43)
no_augs_400 800 (653, 35)
no_augs_400 1600 (653, 56)
no_augs_400 3200 (653, 65)
no_backgrounds 400 (653, 17)
no_backgrounds 800 (653, 18)
no_backgrounds 1600 (653, 33)
no_backgrounds 3200 (653, 35)
extreme_augs_300 400 (653, 38)
extreme_augs_300 800 (653, 36)
extreme_augs_300 1600 (653, 54)
extreme_augs_300 3200 (653, 56)


In [22]:
for model in ['no_augs_400', 'no_backgrounds', '100_talkers', 'extreme_augs_300']:
    tester = np.load(src_dir + f'{model}/arch0_0000/EVAL_220_highpass_activation_block6_relu.npy')
    with open(src_dir + f'{model}/arch0_0000/EVAL_220_highpass.json') as f:
        params = json.load(f)['setting']
    indices = {param: np.array(params) == param for param in sorted(set(params))}
    dims = {}
#     vecs = {}
    for param, inds in indices.items():
        reduced = pca.fit_transform(np.mean(tester[inds], axis=2).reshape((tester[inds].shape[0], tester[inds].shape[1]*tester[inds].shape[3])))
        print(model, param, reduced.shape)
        dims[param] = reduced.shape[1]
        vecs[param] = reduced
    json.dump(dims, open(f'/om2/user/amagaro/word_recognition_project/apmth220_project/{model}_highpass_dims.json', 'w'))
#     json.dump(vecs, open(f'/om2/user/amagaro/word_recognition_project/apmth220_project/{model}_highpass_vecs.json', 'w'))

no_augs_400 400 (653, 234)
no_augs_400 800 (653, 183)
no_augs_400 1600 (653, 111)
no_augs_400 3200 (653, 48)
no_augs_400 6400 (653, 4)
no_backgrounds 400 (653, 273)
no_backgrounds 800 (653, 245)
no_backgrounds 1600 (653, 206)
no_backgrounds 3200 (653, 95)
no_backgrounds 6400 (653, 4)
extreme_augs_300 400 (653, 221)
extreme_augs_300 800 (653, 205)
extreme_augs_300 1600 (653, 176)
extreme_augs_300 3200 (653, 97)
extreme_augs_300 6400 (653, 2)


In [6]:
for block in [6]:
    print('block', block)
    for aug in ['clean']: #['backgrounds-audscene', 'backgrounds-babble', 'backgrounds-music', 'backgrounds-stat_noise', 'backgrounds-mod_noise']:
        print(aug)
        for model in ['100_talkers']:
            fn = f'EVAL_220_all_{aug}'
            tester = np.load(src_dir + f'{model}/arch0_0000/{fn}_activation_block{block}_relu.npy')
            with open(src_dir + f'{model}/arch0_0000/{fn}.json') as f:
                params = json.load(f)['setting']
            indices = {param: np.array(params) == param for param in sorted(set(params))}
#             print(indices)
#             print(np.nanmean(np.isfinite(tester)[indices[1]]))
            dims = {}
        #     vecs = {}
            for param, inds in indices.items():
                reduced = pca.fit_transform(np.mean(tester[inds], axis=2).reshape((tester[inds].shape[0], tester[inds].shape[1]*tester[inds].shape[3])))
                print(model, param, reduced.shape)
                dims[param] = reduced.shape[1]
    #             vecs[param] = reduced
            json.dump(dims, open(f'/om2/user/amagaro/word_recognition_project/apmth220_project/{model}_{aug}_block{block}_dims.json', 'w'))
#         json.dump(vecs, open(f'/om2/user/amagaro/word_recognition_project/apmth220_project/{model}_highpass_vecs.json', 'w'))
# for block in [6, 4, 1]:
#     print('block', block)
#     for aug in ['backgrounds-audscene', 'backgrounds-babble', 'backgrounds-music', 'backgrounds-stat_noise', 'backgrounds-mod_noise']:
#         print(aug)
#         for model in ['100_talkers']: #, 'no_augs_400', 'no_backgrounds', 'extreme_augs_300']:
#             if aug == 'whispered':
#                 fn = f'EVAL_220_all_{aug}2'
#             else:
#                 fn = f'EVAL_220_all_{aug}'
#             tester = np.load(src_dir + f'{model}/arch0_0000/{fn}_activation_block{block}_relu.npy')
#             with open(src_dir + f'{model}/arch0_0000/{fn}.json') as f:
#                 params = json.load(f)['setting']
#             indices = {param: np.array(params) == param for param in sorted(set(params))}
#             dims = {}
#         #     vecs = {}
#             for param, inds in indices.items():
#                 reduced = pca.fit_transform(np.mean(tester[inds], axis=2).reshape((tester[inds].shape[0], tester[inds].shape[1]*tester[inds].shape[3])))
#                 print(model, param, reduced.shape)
#                 dims[param] = reduced.shape[1]
#     #             vecs[param] = reduced
#             json.dump(dims, open(f'/om2/user/amagaro/word_recognition_project/apmth220_project/{model}_{aug}_block{block}_dims.json', 'w'))
    #     json.dump(vecs, open(f'/om2/user/amagaro/word_recognition_project/apmth220_project/{model}_highpass_vecs.json', 'w'))
# for block in [2]:
#     print('block', block)
#     for aug in ['bandpass']:
#         print(aug)
#         for model in ['extreme_augs_300']:
#             if aug == 'whispered':
#                 fn = f'EVAL_220_all_{aug}2'
#             else:
#                 fn = f'EVAL_220_all_{aug}'
#             tester = np.load(src_dir + f'{model}/arch0_0000/{fn}_activation_block{block}_relu.npy')
#             with open(src_dir + f'{model}/arch0_0000/{fn}.json') as f:
#                 params = json.load(f)['setting']
#             indices = {param: np.array(params) == param for param in sorted(set(params))}
#             dims = {}
#         #     vecs = {}
#             for param, inds in indices.items():
#                 reduced = pca.fit_transform(np.mean(tester[inds], axis=2).reshape((tester[inds].shape[0], tester[inds].shape[1]*tester[inds].shape[3])))
#                 print(model, param, reduced.shape)
#                 dims[param] = reduced.shape[1]
#     #             vecs[param] = reduced
#             json.dump(dims, open(f'/om2/user/amagaro/word_recognition_project/apmth220_project/{model}_{aug}_block{block}_dims.json', 'w'))
#     #     json.dump(vecs, open(f'/om2/user/amagaro/word_recognition_project/apmth220_project/{model}_highpass_vecs.json', 'w'))    
# for block in [5, 3, 2]:
#     print('block', block)
#     for aug in ['backgrounds-audscene', 'backgrounds-babble', 'backgrounds-music', 'backgrounds-stat_noise', 'backgrounds-mod_noise']:
#         print(aug)
#         for model in ['100_talkers', 'no_augs_400', 'no_backgrounds', 'extreme_augs_300']:
#             if aug == 'whispered':
#                 fn = f'EVAL_220_all_{aug}2'
#             else:
#                 fn = f'EVAL_220_all_{aug}'
#             tester = np.load(src_dir + f'{model}/arch0_0000/{fn}_activation_block{block}_relu.npy')
#             with open(src_dir + f'{model}/arch0_0000/{fn}.json') as f:
#                 params = json.load(f)['setting']
#             indices = {param: np.array(params) == param for param in sorted(set(params))}
#             dims = {}
#         #     vecs = {}
#             for param, inds in indices.items():
#                 reduced = pca.fit_transform(np.mean(tester[inds], axis=2).reshape((tester[inds].shape[0], tester[inds].shape[1]*tester[inds].shape[3])))
#                 print(model, param, reduced.shape)
#                 dims[param] = reduced.shape[1]
#     #             vecs[param] = reduced
#             json.dump(dims, open(f'/om2/user/amagaro/word_recognition_project/apmth220_project/{model}_{aug}_block{block}_dims.json', 'w'))
#     #     json.dump(vecs, open(f'/om2/user/amagaro/word_recognition_project/apmth220_project/{model}_highpass_vecs.json', 'w'))    

block 6
clean
100_talkers 0 (653, 241)


In [10]:
reduced = pca.fit_transform(tester.reshape((tester.shape[0], np.prod(tester.shape[1:]))))

In [8]:
reduced = pca.fit_transform(np.mean(tester, axis=-1).reshape((tester.shape[0], np.prod(tester.shape[1:3]))))

In [11]:
reduced.shape

(653, 438)

In [16]:
a = np.array([1, 2, 1, 4, 6])
b = np.array([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]).T
b[a == 1]

array([[1, 6],
       [3, 8]])

In [17]:
reduced = pca.fit_transform(np.mean(tester[1], axis=-1))

In [18]:
reduced.shape

(25, 3)

In [19]:
reduced

array([[-4.12768291, -1.36804589, -1.04797941],
       [-5.23072658, -1.58492876, -1.04468824],
       [-2.62942648, -1.39517815, -1.05101728],
       [ 2.4276053 , -0.80410504, -1.01304252],
       [ 7.45120306, -0.19877746, -0.88099071],
       [ 8.84438714,  0.45698703, -0.65784608],
       [ 9.75087207,  0.21963563, -0.6913377 ],
       [ 5.88373402, -1.32244138, -0.82727062],
       [-0.29942266, -2.09277943, -0.53190184],
       [-2.84394499, -1.84455202,  0.07553236],
       [-2.77558717, -1.2549746 ,  0.71133402],
       [-0.08644776, -0.58752962,  1.94479296],
       [ 2.21236937,  0.04681718,  2.82059227],
       [ 1.15594129, -0.09585097,  2.67502782],
       [-0.05372158, -0.29458647,  1.6937615 ],
       [ 0.12825345, -0.25029849,  0.50814259],
       [-1.01288425,  0.36358417,  0.09302255],
       [-1.73113881,  1.0377508 ,  0.06722401],
       [-1.48495599,  2.11767449, -0.14530238],
       [-1.25034271,  2.84131045, -0.53414312],
       [-1.33713451,  2.61384077, -0.498

In [24]:
for i in range(100):
    reduced = pca.fit_transform(np.mean(tester[i], axis=-1))
    print(reduced.shape)

(25, 3)
(25, 3)
(25, 4)
(25, 3)
(25, 4)
(25, 5)
(25, 3)
(25, 4)
(25, 4)
(25, 5)
(25, 5)
(25, 4)
(25, 4)
(25, 3)
(25, 3)
(25, 4)
(25, 4)
(25, 3)
(25, 3)
(25, 4)
(25, 3)
(25, 4)
(25, 4)
(25, 4)
(25, 4)
(25, 4)
(25, 3)
(25, 4)
(25, 3)
(25, 4)
(25, 4)
(25, 3)
(25, 4)
(25, 3)
(25, 4)
(25, 4)
(25, 3)
(25, 4)
(25, 4)
(25, 5)
(25, 5)
(25, 4)
(25, 4)
(25, 4)
(25, 4)
(25, 3)
(25, 3)
(25, 3)
(25, 4)
(25, 4)
(25, 4)
(25, 3)
(25, 4)
(25, 4)
(25, 4)
(25, 4)
(25, 4)
(25, 3)
(25, 5)
(25, 4)
(25, 4)
(25, 4)
(25, 4)
(25, 5)
(25, 3)
(25, 4)
(25, 5)
(25, 4)
(25, 4)
(25, 3)
(25, 3)
(25, 3)
(25, 4)
(25, 5)
(25, 4)
(25, 4)
(25, 4)
(25, 4)
(25, 3)
(25, 4)
(25, 4)
(25, 4)
(25, 4)
(25, 4)
(25, 4)
(25, 3)
(25, 4)
(25, 4)
(25, 4)
(25, 3)
(25, 4)
(25, 5)
(25, 5)
(25, 4)
(25, 4)
(25, 3)
(25, 3)
(25, 4)
(25, 4)
(25, 4)


In [5]:
tester6 = np.load(src_dir + 'no_augs_400/arch0_0000/EVAL_220_mod_lowpass-specmod_activation_block6_relu.npy')

In [9]:
pca = PCA(n_components=0.8, svd_solver='full')
reduced = pca.fit_transform(tester6.reshape(tester6.shape[0], np.prod(tester6.shape[1:])))
print(reduced.shape)

(3265, 653)


In [27]:
tester6.shape

(3265, 13, 53, 512)

In [32]:
np.mean(tester6, axis=(1,2,3)).shape

(3265,)

In [6]:
np.prod

<function numpy.prod(a, axis=None, dtype=None, out=None, keepdims=<no value>, initial=<no value>, where=<no value>)>